# What's the state of indoor agriculture in the US?

Having worked in food distribution for a decade, I probably spend more time thinking about the logistics of food than most people do. When I eat, it's hard for me *not to* imagine where my food came from how it reached me. Still, it wasn't until the pandemic landed in New York City that I'd ever given serious consideration to the possiblity that I might lose access to fresh food. The combination of fear and free time is powerful. 

I found my interest in gardening, long dormant, reawakening virtually overnight. The ability to grow a few vegetables at home provided a small measure of comfort. It was really the knowledge, though, that food was being grown on a commercial scale just blocks from my home that was able to nudge my mind closer to ease. For years, I'd happily been paying a premium for hydroponically grown greens based on quality alone. But, as the pandemic gained momentum and the city locked down, indoor farms - once a source of luxury for me - became a symbol of hope. 

Sure the world was a mess, but we might just be able to dig ourselves out of it.

## Examining population density and the emergence of indoor farming

Once I became attuned, it seemed indoor farms began cropping up left and right. Apparently, the industry was quite a bit larger than I'd realized. Looking to understand the landscape better, the question of *where* exactly these farms were located seemed like a good starting point.

So, what kinds of places become home to indoor farms? Since there are a lot of ways to characterize a place, I decided to narrow my initial scope to population density using data from the most recent US Census, 2020. My own neighborhood can very significantly in the span of a block or two, so I wanted to work with relatively granular population data. While census *blocks* are actually the smallest geographic unit for which census data is gathered, at the time I began this project, the 2020 data at this level had not yet been released. Data at the census tract level, however, was available and I decided it would suffice. 

## Taming the data beast

The [US Decennial Census of Population and Housing](https://www.census.gov/programs-surveys/decennial-census/data/datasets.html) provides a very rich source of raw data, but it can be challenging to decipher and synthesize into usable features. This set of project notebooks walks through the data cleaning and analysis techniques I applied towards my goal of contributing to the understanding of the indoor farming industry and its role in overall food security. The "Data Sources" section of each notebook details the individual files used and the specific steps taken to access them. 

In [1]:
# os and patlib modules used to make it easier to refer to project folders 

import os, pathlib
base_dir = pathlib.Path(os.getcwd()).parent
data_archive_dir = os.path.join(base_dir, "data_archive")
clean_data_dir = os.path.join(data_archive_dir, "clean")
data_dir = os.path.join(base_dir, "data")
shapes_dir = os.path.join(data_dir,"shapes")
json_dir = os.path.join(data_dir,"geojson")
util_dir = os.path.join(data_dir,"util")

## The 50,000 foot view

In [2]:
import pandas as pd
from keplergl import KeplerGl

In [3]:
# import CSV version of GEO DataFrame of NY State Census Tracts 
clean_geodf_tract_ny_file_csv = os.path.join(clean_data_dir,'tl_2020_36_tract20.csv')
geodf_tract_ny = pd.read_csv(clean_geodf_tract_ny_file_csv, index_col=0)
geodf_tract_ny.head(3)

# import CSV version GEO DataFrame of NY State Places
clean_geodf_place_ny_file_csv = os.path.join(clean_data_dir,'geodf_place_ny.csv')
geodf_place_ny = pd.read_csv(clean_geodf_place_ny_file_csv, index_col=0)
geodf_place_ny.head(3)

# make separate GEO DataFrame of the "Place" New York City
geodf_place_nyc = geodf_place_ny[geodf_place_ny['Place FIPS'] == 51000]
geodf_place_nyc

# I want this part to show the outline around NYC
from keplergl import KeplerGl
ny_map = KeplerGl(height=400, show_docs=False)
ny_map.add_data(data=geodf_place_nyc, name='New York City') 
ny_map.add_data(data=geodf_tract_ny)
ny_map.add_data(data=geodf_place_ny)


# I want to add an outline of NYC
# geodf_place_nyc["geometry"].explode().iloc[7]
# geodf_place_nyc["geometry"].boundary.plot()
ny_map

KeplerGl(data={'New York City': {'index': ['New York'], 'columns': ['State', 'State FIPS', 'Place', 'Place FIP…

## Step by step
* [Data Cleanup: US Census population data]()
* Data Cleanup: GEOID tables
* Geographic data: Translating shapefiles to GEO DataFrames
* Mapping NY State: Population Density
* Mapping New York City: Indoor Farms (coming soon...)